<a href="https://colab.research.google.com/github/himanshu03vsk/Relu_coding_round/blob/master/Internship_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below code does iteratre through the csv file and extracts its price, name, image and detail. I observed few things.
<ul>
<li> The two different country code have a different class naming strategy, which causes the code to break. given more time I would have been able to solve this.
<li>There is some kind of encoding issue which causes whitespaces to not be stripped when i call `strip()` method which is reflected in the json file
<li> I had a bit confusion of do i print the wall time or the cpu time for individual iteration or whole dataframe. So i went ahead and printed the wall time for both cases.



In [ ]:
!pip3 install Scrapy

In [ ]:
!pip3 install wget

In [ ]:
!wget --no-check-certificate -O amazon_scraping.csv 'https://docs.google.com/spreadsheets/d/1BZSPhk1LDrx8ytywMHWVpCqbm8URTxTJrIRkD7PnGTM/export?gid=0&format=csv'

In [ ]:
# After checking that the required libs are installed lets look at the main problem
# The task is to simply go to the target endpoint and use the bs4 lib to scrape
# the content required by task

'''Use Selenium or bs4 to Scarpe the following details from the page.
1. Product Title
2. Product Image URL
3. Price of the Product
4. Product Details
If any URL throws Error 404 then print the {URL} not available and skip that
URL.'''

In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
from time import time

df = pd.read_csv("amazon_scraping.csv")

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   id          1000 non-null   int64 
 2   Asin        1000 non-null   object
 3   country     1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB


In [42]:
# lets drop the unnamed :0 column we can use drop funtion or just do it like this
df = df[["id", "Asin", "country"]]
df

,id,Asin,country
0,1,1015,de
1,2424796,1015,fr
2,2,000004458X,de
3,2424797,000004458X,fr
4,3,1002198,de
...,...,...,...
995,2425286,4484924,fr
996,449,4485742,de
997,2425287,4485742,fr
998,450,4486072,de


In [43]:
countries = df.country.values
asins = df.Asin.values
print(len(countries), len(asins))

1000 1000


In [ ]:
# As we can see both arrays have same number of items we can iterate through them, 
# another thing is that we could've iterated over them without assigning them to new variables but in pandas docs it is said to be generally slow
# Program to generate a random number between 0 and 9

# importing the random module
import random

random.randint(0,9)
# The use of multiple try and except blocks are due to the fact that I dont want the whole info to get wasted just becuase one attribute did not captured, so using multiple except
# I retain the info of the product which is found. I can remove it too if necessary
debug = False #used for debugging when bugs are found
start = time()
headers = {
    'User-Agent': f'Mozilla/5.{random.randint(0,9)} (Windows NT 6.{random.randint(0,9)}) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
} ##This header is used to make our request more human because Amazon returned 503 when requested without any headers
country = ""
asin = ""
products_list = []
# url = f"https://www.amazon.{country}/dp/{asin}"
for i in range(1000):
  i_start = time()
  asin = asins[i]
  country = countries[i]
  url = f"https://www.amazon.{country}/dp/{asin}"
  if debug:
    print(url)
  content = requests.get(url, headers=headers)
  print(i, end=" ")
  if content.status_code == 404:
    print("Status code 404 encountered. Skipping...")
    continue
  soup = BeautifulSoup(content.text)
  try:
    product_name = soup.find(id="productTitle").get_text()
  except AttributeError as f:
    product_name = ""
    # print(f
  try:
    image_link = soup.find("img", id=["landingImage", "imgBlkFront"])["src"]
  except (KeyError, TypeError) as f:
    image_link = ""
    # print(f)
  if debug:
    print(image_link)
  #for price we have to go into the div tag containing the price of the product we opened 
  #to find the price of the actual product you have to explore the child nodes of parent price div
  price_div = soup.find("div", {"id":"corePriceDisplay_desktop_feature_div"}) or soup.find("span", {"class":"a-size-base a-color-price a-color-price"} or soup.find_all("span", {"class":"a-color-base"}))
  try:
    if str(type(price_div)) == "<class 'bs4.element.ResultSet'>":
      for curr in price_div:
        if "€" in curr.get_text():
          break
      price= curr.get_text().replace(u'\xa0', u' ')
    else:
      price = price_div.get_text().replace(u'\xa0', u' ')
    spans = price_div.find("span", {"class":"priceToPay"})
    price = spans.find("span", {"class":"a-offscreen"}).get_text()
  except AttributeError as f:
    price=""
    # print(f)
  # for product details find div feature-bullets id and then go inside and find ul and print span text and print the strings of it
  about = soup.find("div", {"id":"feature-bullets"})
  try:
    spans_of_about = about.find_all("span")
  except AttributeError as f:
    spans_of_about = ""
    # print(f)

  product_about = []
  for j in spans_of_about:
    product_about.append(j.get_text().strip())


  details = soup.find("div", {"id":"detailBullets_feature_div","class":"celwidget"})
  try:
    spans_of_detail = details.find_all("span")
  except AttributeError as f:
    spans_of_detail = ""
    # print(f)

  product_details = []
  for k in spans_of_detail:
    product_details.append(k.text.strip())
  # print(f"{product_name}, {image_link},{price}, {product_details[0]}")
  product_dict = {"name":product_name, "img_link":image_link, "price":price, "product_details":product_details, "about":product_about}
  products_list.append(product_dict)
  i_time = time() - i_start
  print(f"Took {i_time}s")




end = time()
time_taken = end - start
print(f"Process took {time_taken:3f}s")

In [199]:
# import json module
import json
  
# list of dictionaries
  
# convert into json
final = json.dumps(products_list, indent=2)
  
#save
with open("products_list_json", "w") as f:
    f.write(final)